In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support
# from tensorflow.keras import layers
import matplotlib.pyplot as plt
# from tensorflow.keras.layers import TextVectorization
import os 

In [2]:
path_200k = 'C:/langchain2/data_medical/200k_abstracts'

In [3]:
def read_lines(filename):
    with open(os.path.join(path_200k, filename), 'r') as f:
        return f.readlines()
train_lines = read_lines('train.txt')
test_lines = read_lines('test.txt')

dev_lines = read_lines('dev.txt')

In [4]:
def preprocessing_abstracts(lines):
    abstract_id = []
    abstract = []
    labeled = []
    for i, line in enumerate(lines):
        if line.isspace():
            continue
        if line.startswith('###'):
            abstract_id.append(line[3:].replace('\n',''))
            j = 0
        else:
            abstract.append({'id': abstract_id[-1].replace('\n',''), 'label': line.split('\t')[0], 'line': line.split('\t')[1], 'num':j })
            j = j+1

    return abstract

In [5]:
train_abstracts = preprocessing_abstracts(train_lines)
test_abstracts = preprocessing_abstracts(test_lines)
dev_abstracts = preprocessing_abstracts(dev_lines)

In [6]:
df_train = pd.DataFrame(train_abstracts)
df_test = pd.DataFrame(test_abstracts)
df_dev = pd.DataFrame(dev_abstracts)

In [7]:
# df_train.to_csv('C:/langchain2/data_medical/train_1.csv', index=False)

In [8]:
df= pd.concat([df_train, df_test, df_dev])

In [9]:
df.iloc[0,0].replace('\n','')

'24491034'

OBJECTIVE=3 ,METHODS=2,RESULTS=4,CONCLUSIONS=1,BACKGROUND=0

how to rename these(0,3,2,4,1) pandas collum sequentyaly to (BACKGROUND=0,OBJECTIVE=3,METHODS=2,RESULTS=4,CONCLUSIONS=1) 

In [10]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [11]:
# Group by id and label, then create a new column containing the concatenated lines
df = df.groupby(['id', 'label'])['line'].apply(lambda x: ' '.join(x)).reset_index()

# Pivot the table to get the labels as columns
df = df.pivot(index='id', columns='label', values='line').reset_index()

# Rename the index column to 'id'
df.index.name = 'id'

# Preview the resulting dataframe
print(df.head())

label        id                                                  0   
id                                                                   
0      10021304                                                NaN  \
1      10021470  Short-term administration of growth hormone to...   
2      10021716                                                NaN   
3      10021717                                                NaN   
4      10021951                                                NaN   

label                                                  1   
id                                                         
0      Sequential combination of platinum-based chemo...  \
1      Long-term administration of growth hormone to ...   
2      Our data confirm that prophylaxis of RDS with ...   
3      Even if the study was terminated before term ,...   
4      Spinal and epidural anaesthesia with lidocaine...   

label                                                  2   
id                          

In [12]:
df = df.rename(columns={0: 'BACKGROUND', 1: 'CONCLUSIONS', 2: 'METHODS', 3: 'OBJECTIVE', 4: 'RESULTS'})
df

label,id,BACKGROUND,CONCLUSIONS,METHODS,OBJECTIVE,RESULTS
id,,,,,,
0,10021304,NaN,Sequential combination of platinum-based chemo...,Between 1985 and 1992 64 patients with radical...,The aim of the study was to evaluate the effec...,The relapse-free and overall survival rates of...
1,10021470,Short-term administration of growth hormone to...,Long-term administration of growth hormone to ...,We studied 121 children with idiopathic short ...,NaN,In the 80 children who have reached adult heig...
2,10021716,NaN,Our data confirm that prophylaxis of RDS with ...,Babies with gestational age 28-33 wks were ran...,To evaluate the impact of administration of su...,"53 babies were analyzed , and 28 were given pr..."
3,10021717,NaN,"Even if the study was terminated before term ,...",Two neonatal intensive care units ( NICU ) in ...,To show if surfactant applied in different soc...,"Due to logistic , practical and social-politic..."
4,10021951,NaN,Spinal and epidural anaesthesia with lidocaine...,Ten healthy volunteers were randomly assigned ...,In this study we sought to determine if and wh...,"At all times , except at time = 0 during spina..."
...,...,...,...,...,...,...
195649,9989713,Groin pain is common among athletes .\n A majo...,AT with a programme aimed at improving strengt...,68 athletes with long-standing ( median 40 wee...,NaN,23 patients in the AT group and four in the PT...
195650,9989716,Reports of mild hypocortisolism in chronic fat...,In some patients with chronic fatigue syndrome...,"In a randomised crossover trial , we screened ...",NaN,None of the patients dropped out .\n Compared ...
195651,9989959,Unfractionated heparin is used widely ; howeve...,These results suggest that the AutoHep system ...,NaN,NaN,We developed and tested a prototype of an auto...


In [13]:
new_order = ['BACKGROUND', 'OBJECTIVE', 'METHODS', 'RESULTS', 'CONCLUSIONS']
df = df[new_order].reset_index(drop=True)


In [14]:
df

label,BACKGROUND,OBJECTIVE,METHODS,RESULTS,CONCLUSIONS
0,NaN,The aim of the study was to evaluate the effec...,Between 1985 and 1992 64 patients with radical...,The relapse-free and overall survival rates of...,Sequential combination of platinum-based chemo...
1,Short-term administration of growth hormone to...,NaN,We studied 121 children with idiopathic short ...,In the 80 children who have reached adult heig...,Long-term administration of growth hormone to ...
2,NaN,To evaluate the impact of administration of su...,Babies with gestational age 28-33 wks were ran...,"53 babies were analyzed , and 28 were given pr...",Our data confirm that prophylaxis of RDS with ...
3,NaN,To show if surfactant applied in different soc...,Two neonatal intensive care units ( NICU ) in ...,"Due to logistic , practical and social-politic...","Even if the study was terminated before term ,..."
4,NaN,In this study we sought to determine if and wh...,Ten healthy volunteers were randomly assigned ...,"At all times , except at time = 0 during spina...",Spinal and epidural anaesthesia with lidocaine...
...,...,...,...,...,...
195649,Groin pain is common among athletes .\n A majo...,NaN,68 athletes with long-standing ( median 40 wee...,23 patients in the AT group and four in the PT...,AT with a programme aimed at improving strengt...
195650,Reports of mild hypocortisolism in chronic fat...,NaN,"In a randomised crossover trial , we screened ...",None of the patients dropped out .\n Compared ...,In some patients with chronic fatigue syndrome...
195651,Unfractionated heparin is used widely ; howeve...,NaN,NaN,We developed and tested a prototype of an auto...,These results suggest that the AutoHep system ...
195652,The Lyon Diet Heart Study is a randomized seco...,NaN,NaN,Three composite outcomes ( COs ) combining eit...,The protective effect of the Mediterranean die...


In [15]:
df['BACKGROUND'] = df['BACKGROUND'].str.replace('\n', ' ')
df['OBJECTIVE'] = df['OBJECTIVE'].str.replace('\n', ' ')
df['METHODS'] = df['METHODS'].str.replace('\n', ' ')
df['RESULTS'] = df['RESULTS'].str.replace('\n', ' ')
df['CONCLUSIONS'] = df['CONCLUSIONS'].str.replace('\n', ' ')


In [16]:
df.to_csv('C:/langchain2/data_medical/full_data.csv', index=False)

In [17]:
df

label,BACKGROUND,OBJECTIVE,METHODS,RESULTS,CONCLUSIONS
0,NaN,The aim of the study was to evaluate the effec...,Between 1985 and 1992 64 patients with radical...,The relapse-free and overall survival rates of...,Sequential combination of platinum-based chemo...
1,Short-term administration of growth hormone to...,NaN,We studied 121 children with idiopathic short ...,In the 80 children who have reached adult heig...,Long-term administration of growth hormone to ...
2,NaN,To evaluate the impact of administration of su...,Babies with gestational age 28-33 wks were ran...,"53 babies were analyzed , and 28 were given pr...",Our data confirm that prophylaxis of RDS with ...
3,NaN,To show if surfactant applied in different soc...,Two neonatal intensive care units ( NICU ) in ...,"Due to logistic , practical and social-politic...","Even if the study was terminated before term ,..."
4,NaN,In this study we sought to determine if and wh...,Ten healthy volunteers were randomly assigned ...,"At all times , except at time = 0 during spina...",Spinal and epidural anaesthesia with lidocaine...
...,...,...,...,...,...
195649,Groin pain is common among athletes . A major...,NaN,68 athletes with long-standing ( median 40 wee...,23 patients in the AT group and four in the PT...,AT with a programme aimed at improving strengt...
195650,Reports of mild hypocortisolism in chronic fat...,NaN,"In a randomised crossover trial , we screened ...",None of the patients dropped out . Compared w...,In some patients with chronic fatigue syndrome...
195651,Unfractionated heparin is used widely ; howeve...,NaN,NaN,We developed and tested a prototype of an auto...,These results suggest that the AutoHep system ...
195652,The Lyon Diet Heart Study is a randomized seco...,NaN,NaN,Three composite outcomes ( COs ) combining eit...,The protective effect of the Mediterranean die...


In [19]:
# import pandas as pd
# import torch
# from transformers import GPT2Tokenizer, GPT2Model


# # Concatenate the text columns and replace '\n' with a space
# df['input_text'] = df['BACKGROUND'].fillna('') + ' ' + df['OBJECTIVE'].fillna('') + ' ' + df['RESULTS'].fillna('') + ' ' + df['CONCLUSIONS'].fillna('')
# df['input_text'] = df['input_text'].str.replace('\n', ' ')

# # Load the GPT-2 model and tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')

# # Set the maximum length of each chunk
# max_len = 512

# # Process each row in the dataset
# for index, row in df.iterrows():
#     # Split the input text into chunks of fixed length
#     input_text = row['input_text']
#     chunks = [input_text[i:i+max_len] for i in range(0, len(input_text), max_len)]

#     # Process each chunk separately and concatenate the output
#     output = ''
#     for chunk in chunks:
#         input_ids = tokenizer.encode(chunk, return_tensors='pt')
#         with torch.no_grad():
#             model_output = model(input_ids)[0]
#         output += tokenizer.decode(model_output[0, -max_len:], skip_special_tokens=True)

#     # Store the output back into the dataframe
#     df.at[index, 'output_text'] = output

# # Save the updated dataframe
# df.to_csv('your_output.csv', index=False)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
# Split the train data into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
train_df.to_csv('C:/langchain2/data_medical/organised_1/train_df.csv', index=False)
test_df.to_csv('C:/langchain2/data_medical/organised_1/test_df.csv', index=False)
val_df.to_csv('C:/langchain2/data_medical/organised_1/val_df.csv', index=False)

In [ ]:
# Print the shape of the resulting dataframes
print("Train set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("Test set shape:", test_df.shape)

In [ ]:
train_df[:30]

AFTER TRAIN TEST VAL SPLIT

In [ ]:
import pandas as pd 

In [ ]:
train_df = pd.read_csv('C:/langchain2/data_medical/organised_1/train_df.csv')
val_df = pd.read_csv('C:/langchain2/data_medical/organised_1/val_df.csv')
test_df = pd.read_csv('C:/langchain2/data_medical/organised_1/test_df.csv')



end#############################################

In [ ]:
train_df

In [ ]:
model_base = Pipeline([
    ("tf-idf", TfidfVectorizer()),
    ("nb", MultinomialNB())
    
])
model_base.fit(df_train['line'], df_train['label'])
model_base.score(df_test['line'], df_test['label'])
'''
l = []
for i in range(0,df_train['line'].shape[0]):
    l.append(len(df_train['line'].to_list()[i].split()))
    
   ''' 
deep_tect_vectorizer = TextVectorization(output_sequence_length = 25)
deep_tect_vectorizer.adapt(df_train['line'])
len(deep_tect_vectorizer.get_vocabulary())

In [ ]:
from torchtext import Field, TabularDataset, BucketIterator

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


# Define the PyTorch model
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, (hidden, cell) = self.rnn(embedded)
        hidden = self.dropout(hidden[-1, :])
        return self.fc(hidden)

# Define the TorchText fields
TEXT = Field(tokenize='spacy', tokenizer_language='en_core_web_sm', include_lengths=True)
LABEL = Field(dtype=torch.float)

# Load the data into a TabularDataset
train_data, test_data = TabularDataset.splits(path='.', train='train.csv', test='test.csv', format='csv', fields=[('text', TEXT), ('label', LABEL)])

# Build the vocabulary
TEXT.build_vocab(train_data, max_size=25000)

# Define the iterators
train_iterator, test_iterator = BucketIterator.splits((train_data, test_data), batch_size=64, sort_within_batch=True, sort_key=lambda x: len(x.text), device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# Define the PyTorch model and optimizer
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
NUM_LAYERS = 2
DROPOUT = 0.5
model = TextClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, DROPOUT)
optimizer = optim.Adam(model.parameters())

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Train the model
model.train()
for epoch in range(10):
    for batch in train_iterator:
        text, text_lengths = batch.text
        labels = batch.label
        optimizer.zero_grad()
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch in test_iterator:
        text, text_lengths = batch.text
        labels = batch.label
        predictions = model(text).squeeze(1)
        predicted_labels = torch.sigmoid(predictions) >= 0.5
        correct += (predicted_labels == labels).sum().item()
        total += labels.size(0)
accuracy = correct / total
print(f'Accuracy: {accuracy}')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

# Load the spacy English language model
spacy_en = spacy.load('en_core_web_sm')

# Define a tokenizer function using spacy
def tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

# Define a custom dataset class
class TextDataset(Dataset):
    def __init__(self, data_path, tokenizer):
        self.data = []
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                text, label = line.strip().split(',')
                self.data.append((text, int(label)))
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        text, label = self.data[index]
        tokenized_text = self.tokenizer(text)
        return tokenized_text, label

# Define a collate function to pad sequences
def collate(batch):
    texts = [torch.LongTensor(item[0]) for item in batch]
    labels = [item[1] for item in batch]
    texts = pad_sequence(texts, batch_first=True)
    return texts, torch.FloatTensor(labels)

# Define the PyTorch model
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, (hidden, cell) = self.rnn(embedded)
        hidden = self.dropout(hidden[-1, :])
        return self.fc(hidden)

# Load the data into a dataset and create a data loader
train_dataset = TextDataset('train.csv', tokenizer)
test_dataset = TextDataset('test.csv', tokenizer)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=collate)

# Build the vocabulary
vocab = set()
for text, label in train_dataset:
    vocab.update(text)
vocab_size = len(vocab)
print(f'Vocabulary size: {vocab_size}')

# Define the PyTorch model and optimizer
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
NUM_LAYERS = 2
DROPOUT = 0.5
model = TextClassifier(vocab_size, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, DROPOUT)
optimizer = optim.Adam(model.parameters())

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Train the model
model.train()
for epoch in range(10):
    for batch in train_loader:
        texts, labels = batch
        optimizer.zero_grad()
        predictions = model(texts).squeeze(1)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch in test_iterator:
        text, text_lengths = batch.text
        labels = batch.label
        predictions = model(text).squeeze(1)
        predicted_labels = torch.sigmoid(predictions) >= 0.5
        correct += (predicted_labels == labels).sum().item()
        total += labels.size(0)
accuracy = correct / total
print(f'Accuracy: {accuracy}')